In [ ]:
# TRAINING ENSEMBLE MULTI-MODELS
import sys
import pandas as pd
from utils.preprocess import get_data_generators
from ai_models.ensemble_model import run_ensemble_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Prepare data generators
train_generator, validation_generator, test_generator, label_encoder, class_weights = get_data_generators()
# Run the ensemble model
run_ensemble_model(train_generator, validation_generator, test_generator, class_weights)

In [ ]:
# TRAINING SINGLE EFFICIENTNETB3 MODEL
import sys
import pandas as pd
from utils.preprocess import get_data_generators
from ai_models.efficientnet_model import train_efficientnetb3, evaluate_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Load data generators
train_gen, val_gen, test_gen, label_enc, class_wts = get_data_generators()

# Train EfficientNetB3
model, history = train_efficientnetb3(train_gen, val_gen, class_wts)

# Evaluate on Test Set
evaluate_model(model, test_gen)


########## GETTING DATA GENERATORS ##########

Loading metadata...


########## TOP FIVE SAMPLE DATA ##########
Metadata:      lesion_id      image_id   dx dx_type   age   sex localization  \
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp   
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp   
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp   
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp   
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear   

                                                path  
0  D:\skin_disease_detection\backend\data\Ham1000...  
1  D:\skin_disease_detection\backend\data\Ham1000...  
2  D:\skin_disease_detection\backend\data\Ham1000...  
3  D:\skin_disease_detection\backend\data\Ham1000...  
4  D:\skin_disease_detection\backend\data\Ham1000...  


########## LABELS ##########
Encoding labels...
Labels encoded. Unique classes: ['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']


c:\Users\Purve\anaconda3\envs\skn-dis-env-2\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
 44/188 ━━━━━━━━━━━━━━━━━━━━ 4:43 2s/step - accuracy: 0.3470 - loss: 1.8077

In [ ]:
# CHECKING ENSEMBLE MULTI-MODEL PREDICTIONS
import sys
import numpy as np
from tensorflow.keras.models import load_model
from joblib import load
from tensorflow.keras.preprocessing.image import load_img, img_to_array

sys.path.append(r'D:\skin_disease_detection\trained_models')

# Load base models
resnet_model = load_model(r"D:\skin_disease_detection\trained_models\ResNet50_model.h5")
print("ResNet50 model loaded successfully!")
efficientnet_model = load_model(r"D:\skin_disease_detection\trained_models\EfficientNetB0_model.h5")
print("EfficientNet model loaded successfully!") 
densenet_model = load_model(r"D:\skin_disease_detection\trained_models\DenseNet121_model.h5")
print("DenseNet model loaded successfully!")

# Load meta-model
meta_model = load(r"D:\skin_disease_detection\trained_models\meta_model.pkl")
print("Meta-model loaded successfully!")

# Load an image
image_path = r"D:\skin_disease_detection\backend\data\Ham10000\HAM10000_images_part_2\ISIC_0029360.jpg"
img = load_img(image_path, target_size=(224, 224))  # Resize as per model input size
print(img)

# Preprocess the image
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Convert single image to batch
img_array = img_array / 255.0 

# Get predictions from the base models
resnet_pred = resnet_model.predict(img_array)
efficientnet_pred = efficientnet_model.predict(img_array)
densenet_pred = densenet_model.predict(img_array)

print("ResNet50 Prediction:", resnet_pred)
print("EfficientNet Prediction:", efficientnet_pred)
print("DenseNet Prediction:", densenet_pred)

# Combine predictions for the meta-model
base_predictions = np.array([resnet_pred, efficientnet_pred, densenet_pred]).reshape(1, -1)

# Get the final prediction from the meta-model
meta_prediction = meta_model.predict(base_predictions)
print("Meta-Model Final Prediction:", meta_prediction)